 # Install/Import Packages and Download/Unzip Dataset

 ---

 The following code installs d3rlpy, imports all necessary dependencies, and downloads/unzips the dataset from Zenodo.

In [ ]:
!pip install d3rlpy==2.6.1 gymnasium==0.29.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827618 sha256=e3d034cc73ff85b0dd55247e33322604221ad7b6917955675ba8997ecb7af7df
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not curr

In [ ]:
from google.colab import files
import numpy as np
import d3rlpy
import os

In [ ]:
!wget -O full_dataset.zip "https://zenodo.org/record/13830810/files/full_dataset.zip?download=1"

--2024-10-10 16:37:49--  https://zenodo.org/record/13830810/files/full_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/13830810/files/full_dataset.zip [following]
--2024-10-10 16:37:49--  https://zenodo.org/records/13830810/files/full_dataset.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 2161582746 (2.0G) [application/octet-stream]
Saving to: ‘full_dataset.zip’

full_dataset.zip    100%[===================>]   2.01G  26.1MB/s    in 2m 44s  

2024-10-10 16:40:33 (12.6 MB/s) - ‘full_dataset.zip’ saved [2161582746/2161582746]



In [ ]:
!unzip full_dataset.zip -d /content/
data_dir = '/content/content/data/data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/data/data/ep_38406.npz  
  inflating: /content/content/data/data/ep_45886.npz  
  inflating: /content/content/data/data/ep_80720.npz  
  inflating: /content/content/data/data/ep_41811.npz  
  inflating: /content/content/data/data/ep_74361.npz  
  inflating: /content/content/data/data/ep_36142.npz  
  inflating: /content/content/data/data/ep_92647.npz  
  inflating: /content/content/data/data/ep_35769.npz  
  inflating: /content/content/data/data/ep_70452.npz  
  inflating: /content/content/data/data/ep_89001.npz  
  inflating: /content/content/data/data/ep_92324.npz  
  inflating: /content/content/data/data/ep_21612.npz  
  inflating: /content/content/data/data/ep_51788.npz  
  inflating: /content/content/data/data/ep_51083.npz  
  inflating: /content/content/data/data/ep_54442.npz  
  inflating: /content/content/data/data/ep_97628.npz  
  inflating: /content/content/data/data/ep_26720.npz  
  inflating: /

# Prepare Dataset for d3rlpy

---

The following code loads all of the data into arrays and creates a d3rlpy dataset.

In [ ]:
total_steps = 5000000

all_observations = np.empty((total_steps, 3, 64, 64), dtype=np.uint8)
all_actions = np.empty((total_steps, 3), dtype=np.float32)
all_rewards = np.empty((total_steps,), dtype=np.float32)
all_terminals = np.empty((total_steps,), dtype=bool)

current_index = 0

for i in range(1, 99523):
    episode_file = os.path.join(data_dir, f'ep_{i}.npz')
    episode_data = np.load(episode_file)

    num_steps = episode_data['terminals'].shape[0]

    all_observations[current_index:current_index + num_steps] = np.transpose(episode_data['observations'], (0, 3, 1, 2))
    all_actions[current_index:current_index + num_steps] = episode_data['actions']
    all_rewards[current_index:current_index + num_steps] = episode_data['rewards']
    all_terminals[current_index:current_index + num_steps] = episode_data['terminals']

    current_index += num_steps

all_observations = all_observations[:current_index]
all_actions = all_actions[:current_index]
all_rewards = all_rewards[:current_index]
all_terminals = all_terminals[:current_index]

In [ ]:
dataset = d3rlpy.dataset.MDPDataset(
    observations=all_observations,
    actions=all_actions,
    rewards=all_rewards,
    terminals=all_terminals,
)

2024-10-10 16:56.08 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]) observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2024-10-10 16:56.08 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2024-10-10 16:56.08 [info     ] Action size has been automatically determined. action_size=3


# Model Training

---

The following code trains a TD3 + behavior cloning algorithm for 50 epochs at 10000 steps per epoch.

In [ ]:
TD3 = d3rlpy.algos.TD3PlusBCConfig(observation_scaler=d3rlpy.preprocessing.PixelObservationScaler(),).create(device='cuda')
TD3.fit(dataset=dataset, n_steps=500000, n_steps_per_epoch=10000)

2024-10-10 16:56.13 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2024-10-10 16:56.13 [info     ] Directory is created at d3rlpy_logs/TD3PlusBC_20241010165613
2024-10-10 16:56.13 [debug    ] Building models...            
2024-10-10 16:56.15 [debug    ] Models have been built.       
2024-10-10 16:56.15 [info     ] Parameters                     params={'observation_shape': [3, 64, 64], 'action_size': 3, 'config': {'type': 'td3_plus_bc', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'pixel', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'act

Epoch 1/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-10-10 17:00.23 [info     ] TD3PlusBC_20241010165613: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.008369894599914551, 'time_algorithm_update': 0.016096764087677, 'critic_loss': 93.82236885137559, 'actor_loss': -2.4616885204315184, 'bc_loss': 0.03657058561388403, 'time_step': 0.024614270520210268} step=10000
2024-10-10 17:00.23 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20241010165613/model_10000.d3


Epoch 2/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-10-10 17:04.26 [info     ] TD3PlusBC_20241010165613: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.008223797225952149, 'time_algorithm_update': 0.015871265625953673, 'critic_loss': 11.852331443214416, 'actor_loss': -2.4799877138614654, 'bc_loss': 0.01696816382277757, 'time_step': 0.024231420373916627} step=20000
2024-10-10 17:04.26 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20241010165613/model_20000.d3


...

Epoch 49/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-10-10 20:18.16 [info     ] TD3PlusBC_20241010165613: epoch=49 step=490000 epoch=49 metrics={'time_sample_batch': 0.008497915077209473, 'time_algorithm_update': 0.01606016035079956, 'critic_loss': 0.6690678251951933, 'actor_loss': -2.492020642662048, 'bc_loss': 0.007979360209126025, 'time_step': 0.024712475061416626} step=490000
2024-10-10 20:18.16 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20241010165613/model_490000.d3


Epoch 50/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-10-10 20:22.21 [info     ] TD3PlusBC_20241010165613: epoch=50 step=500000 epoch=50 metrics={'time_sample_batch': 0.008335050678253174, 'time_algorithm_update': 0.015917618370056154, 'critic_loss': 0.6676474977880716, 'actor_loss': -2.4920443999767303, 'bc_loss': 0.007955600764788687, 'time_step': 0.02440484297275543} step=500000
2024-10-10 20:22.21 [info     ] Model parameters are saved to d3rlpy_logs/TD3PlusBC_20241010165613/model_500000.d3


[(1,
  {'time_sample_batch': 0.008369894599914551,
   'time_algorithm_update': 0.016096764087677,
   'critic_loss': 93.82236885137559,
   'actor_loss': -2.4616885204315184,
   'bc_loss': 0.03657058561388403,
   'time_step': 0.024614270520210268}),
 (2,
  {'time_sample_batch': 0.008223797225952149,
   'time_algorithm_update': 0.015871265625953673,
   'critic_loss': 11.852331443214416,
   'actor_loss': -2.4799877138614654,
   'bc_loss': 0.01696816382277757,
   'time_step': 0.024231420373916627}),
 (3,
  {'time_sample_batch': 0.00825217661857605,
   'time_algorithm_update': 0.015892396545410158,
   'critic_loss': 4.553166068780422,
   'actor_loss': -2.484114922428131,
   'bc_loss': 0.015283993274532259,
   'time_step': 0.024285818433761597}),
 (4,
  {'time_sample_batch': 0.008270167994499207,
   'time_algorithm_update': 0.015890220284461975,
   'critic_loss': 2.2316873225986957,
   'actor_loss': -2.4878032606124876,
   'bc_loss': 0.012165561911650002,
   'time_step': 0.02429697051048279})

In [ ]:
!zip -r td3_model_full_dataset.zip d3rlpy_logs

  adding: d3rlpy_logs/ (stored 0%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/ (stored 0%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_460000.d3 (deflated 23%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_220000.d3 (deflated 23%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_410000.d3 (deflated 23%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_280000.d3 (deflated 23%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_180000.d3 (deflated 22%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_40000.d3 (deflated 15%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_50000.d3 (deflated 15%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/model_190000.d3 (deflated 22%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/params.json (deflated 76%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/time_step.csv (deflated 59%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/time_sample_batch.csv (deflated 60%)
  adding: d3rlpy_logs/TD3PlusBC_20241010165613/mo